In [8]:
from tensorflow.keras.models import load_model

import pandas as pd
import numpy as np

In [4]:
MODEL_DIR = './models/'

In [6]:
model = load_model(MODEL_DIR + 'seq_25k')
model

In [9]:
# Predict matchup
off_id = 201142
def_id = 201939

# Get offense stats
data = pd.read_csv('./stats/2021-22_off_stats.csv')
data = data.add_prefix('OFF_')
data.rename(columns={'OFF_SEASON_ID': 'SEASON_ID'}, inplace=True)
inputs_o = data[data['OFF_PLAYER_ID'] == off_id]

# Get defensive stats
def_data = pd.read_csv('./stats/2021-22_def_stats.csv')
def_data = def_data.fillna(0)
def_data = def_data.add_prefix('DEF_')
def_data.rename(columns={'DEF_SEASON_ID': 'SEASON_ID', 'DEF_DEF_RATING': 'DEF_RATING', 
                     'DEF_DEF_BOXOUTS': 'DEF_BOXOUTS'}, inplace=True)
inputs_d = def_data[def_data['DEF_PLAYER_ID'] == def_id]
print(inputs_d['DEF_PLAYER_NAME'])

# Get both inputs together
inputs_o.drop(columns=['OFF_PLAYER_ID', 'OFF_PTS_PER_100'], inplace=True)
inputs_d.drop(columns=['DEF_PLAYER_ID', 'DEF_PLAYER_NAME', 'SEASON_ID'], inplace=True)
inputs_o.drop(columns=['OFF_POST_TOUCH_PTS', 'OFF_POST_TOUCH_FG_PCT', 'OFF_FGM', 'OFF_PTS', 'OFF_EFF_FG_PCT', 'OFF_OREB', 'OFF_PAINT_TOUCH_FGA', 'OFF_PAINT_TOUCH_FGM', 'OFF_PAINT_TOUCH_TOV', 'OFF_PAINT_TOUCH_PASSES', 'OFF_PAINT_TOUCH_PTS', 'OFF_DREB', 'OFF_FG3M', 'OFF_FTM'], inplace=True)
inputs_d.drop(columns=['DEF_D_FGM', 'DEF_OPP_PTS_PAINT', 'DEF_CONTESTED_SHOTS_2PT', 'DEF_DEFLECTIONS', 'DEF_OPP_PTS_FB', 'DEF_OPP_PTS_2ND_CHANCE', 'DEF_OPP_PTS_OFF_TOV', 'DEF_PCT_PLUSMINUS', 'DEF_G'], inplace=True)

inputs_o = inputs_o.reset_index(drop=True)
inputs_d = inputs_d.reset_index(drop=True)
X_predict = pd.concat([inputs_d, inputs_o], axis=1)

# Normalize inputs
X_seqtest = pd.read_csv('./stats/X.csv')
X_seqtest.drop(columns=['OFF_POST_TOUCH_PTS', 'OFF_POST_TOUCH_FG_PCT', 'OFF_PTS'], inplace=True)
X_seqtest = X_seqtest[X_seqtest['SEASON_ID'] == 22021]

X_predict = (X_predict - X_seqtest.mean()) / X_seqtest.std()

X_predict = X_predict.fillna(0)

X_predict = X_predict.sort_index(axis=1)

model.predict(X_predict.to_numpy())

508    Stephen Curry
Name: DEF_PLAYER_NAME, dtype: object


C:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


array([[25.012611]], dtype=float32)

In [10]:
test = pd.read_csv('./stats/y.csv')
test = test[test['OFF_PLAYER_ID'] == 201939]
test['PLAYER_PTS'].mean()

41.780257704344145